# Parsing Debug Notebook

This somewhat messy notebook makes it easier to debug the parser, because we can just rerun the cells needed to set up the debugging process with various parameters.

In [1]:
import json
import math
import re
from pathlib import Path

import boto3
import pandas as pd

from parse_990_textract.bucket import open_df
from parse_990_textract.filing import create_roadmap, extract_from_roadmap
from parse_990_textract.models import BoundingBox, TableExtractor
from parse_990_textract.parse import create_extractors, find_item, find_pages
from parse_990_textract.postprocessing import clean_filing, clean_f_i, clean_f_ii, clean_f_iii, postprocess
from parse_990_textract.setup import load_extractor_df
from parse_990_textract.table import extract_table_data, find_table_pages, create_tablemap
from parse_990_textract.utils import get_coordinate, get_regex, cluster_words, columnize, cluster_x

In [2]:
bucket = boto3.resource("s3").Bucket("s3-ocr-990s-demo")

In [3]:
validation_data = pd.read_csv("validation_data.csv", index_col="job_id").fillna("")
validation_data.head()

,pdf_key,other_expenses_c_mgmt_general,payments_affiliates_total,other_expenses_d_prog_service,total_revenue,fees_for_lobbying_services_total,other_employee_benefits_fundraising,total_fundraising_expenses,compensation_officers_prog_service,fees_for_mgmt_services_total,...,travel_total,pension_plan_acc_contrib_prog_service,advertising_promotion_mgmt_general,pension_plan_acc_contrib_mgmt_general,compensation_disq_persons_total,other_salaries_wages_mgmt_general,office_expenses_mgmt_general,fees_for_other_services_fundraising,total_number_other_recipient_foreign_orgs_entities,activities_per_region_totals_total_expenditure
job_id,,,,,,,,,,,,,,,,,,,,,
0f908c03383d094f6c1386749189f281d188d5ea0cd64c4e424a5b1aae1650c4,EIN_760733035_YEAR_2009_FORMTYPE_990.pdf,,,33000.0,1415024,,,4503.0,,,...,18307.0,,,,,,,,,
bcc635fa2b088a72666c3d534d0221a1c1294f2974a6a304679654069e5cbcdc,EIN_363235550_YEAR_2009_FORMTYPE_990.pdf,,,15832.0,9899921,,27161.0,597789.0,295722.0,,...,677722.0,34261.0,30,3462.0,,159156.0,48381.0,22350.0,,
f0545c488bd7ab7e775c25cfa7d050a3ff1f84dea9a3371b8fef236bd3d0624b,EIN_223134995_YEAR_2010_FORMTYPE_990.pdf,0.0,,10823.0,3370595,,12776.0,263765.0,148689.0,2250.0,...,197092.0,,127381,,396855.0,103312.0,11723.0,23021.0,,
4f2b05354be9fb6483976694a1ed0494c7c387631c15130bdaaa1a85a1cf4f82,EIN_264320885_YEAR_2009_FORMTYPE_990.pdf,,,,46117,,,669.0,,,...,,,,,,,458.0,,,
1dd5dc37f4e99e27668c769d04d0f72273a6c582d504bf0b88d35d5a4c15c1be,EIN_581943161_YEAR_2009_FORMTYPE_990.pdf,581.0,,0.0,583209,,2521.0,14409.0,298484.0,,...,7233.0,,,,,,4010.0,0.0,,


In [4]:
extractor_df = load_extractor_df("990_extractors.csv")
roadmap_df = load_extractor_df("990_roadmap.csv")
schedule_f_tablemap_df = load_extractor_df("schedule_f_table_roadmap.csv")
schedule_f_table_extractor_df = pd.read_csv("schedule_f_table_extractors.csv")
schedule_f_row_extractor_df = pd.read_csv("schedule_f_row_extractors.csv")

In [5]:
PART_I_HEADER = r"\(a\)\s*Region|\(d\)\s*Activities|\(e\)\s*If activity|\(f\)Total expenditures"
PART_II_HEADER = r"\(b\)\s*IRS code|\(c\)\s*Region|\(d\)\s*Purpose|\(f\)\s*Manner|\(h\)\s*Description"
PART_III_HEADER = r"\(b\)\s*Region|\(e\)\s*Manner of cash|\(h\)\s*Method of va"
PART_I_TABLE_NAME = "Activities per Region"
PART_II_TABLE_NAME = r"Grants to Organizations Outside the United States"
PART_III_TABLE_NAME = "Grants to Individuals Outside the United States"

In [6]:
filing_rows = []
schedule_f_part_i_rows = []
schedule_f_part_ii_rows = []
schedule_f_part_iii_rows = []


for i, job_id in enumerate(validation_data.index.values):
    print(i)
    print(job_id)
    pdf_key = validation_data.at[job_id, "pdf_key"]
    print(pdf_key)
    
    data = open_df(bucket, job_id)
    lines = data.loc[data["BlockType"] == "LINE"]
    words = data.loc[data["BlockType"] == "WORD"]
    page_map = find_pages(lines)
    roadmap = create_roadmap(
        lines, roadmap_df, page_map
    )
    
    row = extract_from_roadmap(
        words, lines, roadmap, extractor_df, page_map
    )
    row = postprocess(row, job_id, pdf_key, clean_filing)
    filing_rows.append(row)
    
    pages = lines.groupby("Page")
    
    part_i_table = extract_table_data(
        pages, lines, words, PART_I_HEADER, PART_I_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_i_table = postprocess(part_i_table, job_id, pdf_key, clean_f_i)
    if part_i_table is not None:
        schedule_f_part_i_rows.append(
            part_i_table
        )
    part_ii_table = extract_table_data(
        pages, lines, words, PART_II_HEADER, PART_II_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_ii_table = postprocess(part_ii_table, job_id, pdf_key, clean_f_ii)
    if part_ii_table is not None:
        schedule_f_part_ii_rows.append(
            part_ii_table
        )
    part_iii_table = extract_table_data(
        pages, lines, words, PART_III_HEADER, PART_III_TABLE_NAME, 
        schedule_f_tablemap_df, schedule_f_table_extractor_df, schedule_f_row_extractor_df,
    )
    part_iii_table = postprocess(part_iii_table, job_id, pdf_key, clean_f_iii)
    if part_iii_table is not None:
        schedule_f_part_iii_rows.append(
            part_iii_table
        )

0
0f908c03383d094f6c1386749189f281d188d5ea0cd64c4e424a5b1aae1650c4
EIN_760733035_YEAR_2009_FORMTYPE_990.pdf


No match for year_formation in L Year of Formation M State of legal domicile
No match for state_of_domicile in M State of legal domicile


1
bcc635fa2b088a72666c3d534d0221a1c1294f2974a6a304679654069e5cbcdc
EIN_363235550_YEAR_2009_FORMTYPE_990.pdf


No match for total_unrelated_biz_revenue in 7a NONE
No match for net_unrelated_biz_taxable_revenue in 7b NONE
No match for benefits_paid_members_expenses in NONE
No match for professional_fundraising_fees_expenses in NONE
No match for grants_foreign_individuals_govt_orgs_total in NONE
No match for benefits_to_members_total in NONE
No match for compensation_disq_persons_total in NONE
No match for fees_for_mgmt_services_total in NONE
No match for fees_for_lobbying_services_total in NONE
No match for fees_for_fundraising_services_total in NONE
No match for fees_for_investment_mgmt_services_total in NONE
No match for royalties_total in NONE
No match for entertainment_travel_govt_officials_total in NONE
No match for interest_total in NONE
No match for payments_affiliates_total in NONE


2
f0545c488bd7ab7e775c25cfa7d050a3ff1f84dea9a3371b8fef236bd3d0624b
EIN_223134995_YEAR_2010_FORMTYPE_990.pdf
3
4f2b05354be9fb6483976694a1ed0494c7c387631c15130bdaaa1a85a1cf4f82
EIN_264320885_YEAR_2009_FORMTYPE_990.pdf
4
1dd5dc37f4e99e27668c769d04d0f72273a6c582d504bf0b88d35d5a4c15c1be
EIN_581943161_YEAR_2009_FORMTYPE_990.pdf
5
5596f51a999ebbd4cb992f490ceaffcddbac9bce532b911997683ae6897c2797
EIN_231352689_YEAR_2009_FORMTYPE_990.pdf


...

<class 'KeyError'>: 'Bottom (i)'
<class 'KeyError'>: 'Bottom (h)'


22
a6529e504df346097da99104a353e977426e018cd5ac33b62cd2dd89c90763c5
EIN_311002913_YEAR_2008_FORMTYPE_990.pdf
23
d1925c2d74adaa3f150ded3ce67dfe7ae3a306f0db0289ad5755a28d801b2b0b
EIN_202408857_YEAR_2010_FORMTYPE_990.pdf


No match for activities_per_region_totals_number_of_offices in Act the
<class 'KeyError'>: 'Bottom (i)'
<class 'KeyError'>: 'Bottom (h)'


24
01d89ee5d14575c1321b2e4d67431d172ba76212b4a266bdaf474275029fd78b
EIN_521830327_YEAR_2009_FORMTYPE_990.pdf


No match for total_fundraising_expenses in b Total fundraising expenses (Part IX, column (D), line 25)
No match for revenue_less_expenses in <43,775.> End of Year


In [7]:
output_data = pd.concat(filing_rows).reset_index(drop=True).set_index("job_id")

In [8]:
output_data.head()

field_name,index,name,address,city,state,zip,website,gross_receipts,year_formation,state_of_domicile,...,total_functional_expense_fundraising,activities_per_region_totals_number_of_offices,activities_per_region_totals_number_of_employees,activities_per_region_totals_total_expenditure,total_number_recipient_foreign_orgs_listed_as_charities,total_number_other_recipient_foreign_orgs_entities,pdf_key,ein,year,filing_id
job_id,,,,,,,,,,,,,,,,,,,,,
0f908c03383d094f6c1386749189f281d188d5ea0cd64c4e424a5b1aae1650c4,0,MAKE WAY PARTNERS INC,PO BOX 26367,BIRMINGHAM,OX,26367,www MAKEWAYPARTNERS ORG,1426217,,,...,4503,,,,,,EIN_760733035_YEAR_2009_FORMTYPE_990.pdf,760733035,2009,760733035_2009
bcc635fa2b088a72666c3d534d0221a1c1294f2974a6a304679654069e5cbcdc,0,THE FEDERALIST SOCIETY FOR LAW AND,"1015 18TH ST., N.W. 425",WASHINGTON,DC,20036,www. FED-SOC.ORG,11033302,1982,IL,...,597789,,,,,,EIN_363235550_YEAR_2009_FORMTYPE_990.pdf,363235550,2009,363235550_2009
f0545c488bd7ab7e775c25cfa7d050a3ff1f84dea9a3371b8fef236bd3d0624b,0,Ron Hutchcraft Ministries Inc,PO Box 400,Harnson,AR,20400,www hutchcraft com,3390,1991,NJ,...,263765,,,,,,EIN_223134995_YEAR_2010_FORMTYPE_990.pdf,223134995,2010,223134995_2010
4f2b05354be9fb6483976694a1ed0494c7c387631c15130bdaaa1a85a1cf4f82,0,FREEDOM 4 24,21430 TIMBERLAKE ROAD STE 101,LYNCHBURG,VA,24502,freedom424 org,61857,2009,VA,...,669,,,,,,EIN_264320885_YEAR_2009_FORMTYPE_990.pdf,264320885,2009,264320885_2009
1dd5dc37f4e99e27668c769d04d0f72273a6c582d504bf0b88d35d5a4c15c1be,0,GEORGIA PUBLIC POLICY FOUNDTION,6100 LAKE FORREST DR,LAKE FORREST,GA,30328,www GPPF org,583209,1991,GA,...,14409,,,,,,EIN_581943161_YEAR_2009_FORMTYPE_990.pdf,581943161,2009,581943161_2009


In [9]:
def clean(x):
    x = str(x)
    x = re.sub(r"\.0\b", "", x)
    x = re.sub("\D", "", x)
    return x

In [10]:
def compare_output(to_validate, to_compare, col):
    return pd.DataFrame(
        {
            "extracted": to_validate.loc[col].loc[
                lambda series: series != to_compare.loc[col]
            ],
            "expected": to_compare.loc[col].loc[
                lambda series: series != to_validate.loc[col]
            ],
        }
    )

In [11]:
to_compare = validation_data.set_index("pdf_key").applymap(clean)
to_validate = output_data[validation_data.columns].set_index("pdf_key").applymap(clean)

for col in to_validate.index:
    validated = compare_output(to_validate, to_compare, col)
    if validated.any().any():
        print(col)
        print(f"{validated.shape[0]} mismatched items.")
        print(validated)
        print("-"*79)

EIN_113489123_YEAR_2008_FORMTYPE_990.pdf
2 mismatched items.
              extracted expected
total_revenue      1017  1017506
travel_total        137    13710
-------------------------------------------------------------------------------
EIN_582248383_YEAR_2009_FORMTYPE_990.pdf
1 mismatched items.
                                                   extracted expected
total_number_other_recipient_foreign_orgs_entities        10         
-------------------------------------------------------------------------------


In [12]:
assert False

AssertionError: 

## BREAK

In [13]:
pd.concat(schedule_f_part_i_rows).head(50)[
    [
        "region", "number_offices", "number_employees", 
        "activities_conducted", "specific_type_activity", 
        "total_expenditures", "pdf_key"
    ]
]

field,region,number_offices,number_employees,activities_conducted,specific_type_activity,total_expenditures,pdf_key
0,Europe,1,0,theological training,offers a Th M degree,6661,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf
1,Totals,1,0,,,6661,EIN_231352689_YEAR_2009_FORMTYPE_990.pdf
0,(1),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
1,(2),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
2,(3),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
3,(4),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
4,(5),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
5,(6),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
6,(7),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf
7,(8),,,,,,EIN_521238301_YEAR_2010_FORMTYPE_990.pdf


In [ ]:
pd.concat(schedule_f_part_i_rows).tail(50)

In [ ]:
pd.concat(schedule_f_part_ii_rows).iloc[:50]

In [ ]:
pd.concat(schedule_f_part_iii_rows).iloc[:50]

In [89]:
table_test_df = open_df(bucket, "67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebdb32d3186b19d6a6fd5")

In [90]:
test_lines = table_test_df.loc[
    table_test_df["BlockType"] == "LINE"
]
test_words = table_test_df.loc[
    table_test_df["BlockType"] == "WORD"
]
test_pages = test_lines.groupby("Page")

In [91]:
HEADER = PART_I_HEADER
NAME = PART_I_TABLE_NAME

In [92]:
table_pages = find_table_pages(
    test_pages["Text"].agg(lambda words: " ".join(words)),
    HEADER,
)

In [93]:
table_pages

Page
26    26
30    30
Name: Page, dtype: int64

In [94]:
TEST_INDEX = 1
TEST_PAGE = 30

In [95]:
tablemaps = pd.DataFrame(
    {
        "page": table_pages,
        "tablemap": table_pages.map(
            lambda page: create_tablemap(
                test_lines, schedule_f_tablemap_df, page, NAME
            ).dropna()
        ),
    }
)

In [97]:
tablemaps["tablemap"].iloc[TEST_INDEX]

,Top,Left,Top_Default,Left_Default
Item,,,,
(a) Region,0.056952,0.063677,0.05,0.0
(b) Number of offices,0.060424,0.167132,0.05,0.0
(c) Number of employees,0.060752,0.254655,0.05,0.0
(d) Activities conducted,0.060764,0.343045,0.05,0.0
(e) Specific type,0.056969,0.480311,0.05,0.0
(f) Total Expenditures,0.056891,0.612973,0.05,0.0
"Schedule F, Part I, Item 3a",0.716738,0.006047,0.00,0.0
Bottom (d),0.077065,0.348286,0.00,0.0
Top Left Corner,0.000000,0.000000,0.00,0.0


In [98]:
row_extractors = schedule_f_row_extractor_df.loc[
    schedule_f_row_extractor_df["table"] == NAME
]
table_data = schedule_f_table_extractor_df.loc[
    schedule_f_table_extractor_df["table"] == NAME
].iloc[0]

In [99]:
rows = tablemaps.assign(
    extractor=tablemaps["tablemap"].map(
        lambda tablemap: TableExtractor(
            header_top_label=table_data["header_top"],
            top_label=table_data["table_top"],
            bottom_label=table_data["table_bottom"],
            tablemap=tablemap,
            fields=row_extractors["field"],
            field_labels=row_extractors["col_left"]
        )
    )
)

In [100]:
test_extractor = rows["extractor"].iloc[TEST_INDEX]

In [101]:
test_extractor.get_col_spans(test_words, TEST_PAGE)

0                    (0.0, 0.1671319603919983)
1    (0.1671319603919983, 0.25465500354766846)
2    (0.25465500354766846, 0.3363614082336426)
3     (0.3363614082336426, 0.4708361327648163)
4     (0.4708361327648163, 0.6129726767539978)
5                    (0.6129726767539978, 1.0)
dtype: object

In [102]:
test_extractor.extract_rows(test_words, TEST_PAGE).head(50)

field,region,number_offices,number_employees,activities_conducted,specific_type_activity,total_expenditures
0,Central merica and the Carbbear,,,Fundraising,,481
1,Central merica the Caribbean,,,Grantmaking,,"303,788"
2,Central merica the anbbea,,,Program services,Academic Support,"20,973"
3,:entral America the :aribbean,,,Program Services,Instruction,"58,431"
4,entral merica the Caribbear,,,Program services,Research and other academic activity,"228,975"
5,Central merica the Taribbean,,,Investments,,
6,East Asia and Pacific,,,Fundraising,,"3,593"
7,East Asia and Pacific,,,Grantmaking,,"1,269,299"
8,East Asia and Pacific,,,rogr services,Service Centers,"118,026"
9,East Asia and Pacific,,,rogram Services,Academic Support,"461,230"


In [103]:
table_words = test_extractor.get_table_words(test_words, TEST_PAGE)

In [104]:
table_words

,BlockType,Confidence,Geometry,Page,PageClassification,Relationships,Text,TextType,Polygon,Height,Left,Top,Right,Bottom,Midpoint_X,Midpoint_Y,Width,Children,Line_No,File
Id,,,,,,,,,,,,,,,,,,,,
6f17b9fd-5937-4bf0-a004-63f84ada93e7,WORD,96.340424,{'BoundingBox': {'Height': 0.00317446910776197...,30,None,None,Central,PRINTED,"[{'X': 0.01150417048484087, 'Y': 0.08279873430...",0.003174,0.011504,0.082799,0.047105,0.085973,0.029305,0.084386,0.035601,None,12,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...
33733a90-daf3-45cd-be33-db2b819aa02c,WORD,66.090393,{'BoundingBox': {'Height': 0.00327668758109211...,30,None,None,merica,PRINTED,"[{'X': 0.051776643842458725, 'Y': 0.0828743427...",0.003277,0.051777,0.082874,0.091312,0.086151,0.071544,0.084513,0.039535,None,12,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...
b6c12310-3590-4462-8871-7b8cb2f8c636,WORD,87.114311,{'BoundingBox': {'Height': 0.00299885566346347...,30,None,None,and,PRINTED,"[{'X': 0.09316416829824448, 'Y': 0.08286353200...",0.002999,0.093164,0.082864,0.111959,0.085862,0.102562,0.084363,0.018795,None,12,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...
580af183-1480-42c9-b94d-1e5382641f7b,WORD,96.795914,{'BoundingBox': {'Height': 0.00291400658898055...,30,None,None,the,PRINTED,"[{'X': 0.11327695846557617, 'Y': 0.08304906636...",0.002914,0.113277,0.083049,0.131671,0.085963,0.122474,0.084506,0.018394,None,12,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...
9cc288c7-b2db-49c7-8812-3ae88c37edae,WORD,88.749054,{'BoundingBox': {'Height': 0.00323019921779632...,30,None,None,Fundraising,PRINTED,"[{'X': 0.3407772183418274, 'Y': 0.082760512828...",0.003230,0.340777,0.082761,0.393868,0.085991,0.367323,0.084376,0.053091,None,12,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215541fd-d73a-4af3-ad80-9e542043cc5b,WORD,87.614494,{'BoundingBox': {'Height': 0.00374788418412208...,30,None,None,"5,575,512",PRINTED,"[{'X': 0.6699190735816956, 'Y': 0.695733010768...",0.003748,0.669919,0.695733,0.726750,0.699481,0.698334,0.697607,0.056830,None,75,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...
0ea0cc52-4e14-468d-8eaa-6e440e864390,WORD,96.324692,{'BoundingBox': {'Height': 0.00321036786772310...,30,None,None,Saharan,PRINTED,"[{'X': 0.03040502592921257, 'Y': 0.70516633987...",0.003210,0.030405,0.705166,0.076196,0.708377,0.053300,0.706772,0.045791,None,76,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...
10d631e0-e89b-4b97-81c8-46375a4340b1,WORD,93.906059,{'BoundingBox': {'Height': 0.00303548178635537...,30,None,None,Africa,PRINTED,"[{'X': 0.07765699923038483, 'Y': 0.70513230562...",0.003035,0.077657,0.705132,0.106436,0.708168,0.092047,0.706650,0.028779,None,76,67217e04d83f69ffbbb461b0a23648b037b4a36d0c5ebd...


In [105]:
word_clusters = cluster_words(table_words, table_words["Height"].min(), "Midpoint_Y")
[" ".join(word.sort_values(by="Left")["Text"].values) for word in word_clusters]

['Central merica and the Fundraising 481',
 'Carbbear',
 'Central merica the Grantmaking 303,788',
 'Caribbean',
 'Central merica the Program services Academic Support 20,973',
 'anbbea',
 ':entral America the Program Services Instruction 58,431',
 ':aribbean',
 'entral merica the Program services Research and other 228,975',
 'Caribbear academic activity',
 'Central merica the Investments',
 'Taribbean',
 'East Asia and Pacific Fundraising 3,593',
 'East Asia and Pacific Grantmaking 1,269,299',
 'East Asia and Pacific rogr services Service Centers 118,026',
 'East Asia and Pacific rogram Services Academic Support 461,230',
 'East Asia and the Pacific Progr services Institutiona support 71,217',
 'East Asia and Pacific Program Services instruction 5,628,477',
 'East Asia and Pacific 44 Progr Services Research other 6,420,156',
 'Academic Activity',
 'East Asia and Pacific Programservices Student services 22,043',
 'East Asia and Pacific Investments',
 'Europe Including Iceland & Fundra

In [106]:
def columnize(word_cluster, col_spans):
    return col_spans.map(
        lambda span: word_cluster.loc[
            (word_cluster["Left"].between(*span, inclusive="left"))
        ]
    )


def get_cluster_coords(cluster):
    cluster_coords = {
        "Left": cluster["Left"].min(),
        "Right": cluster["Right"].max(),
        "Height": cluster["Height"].max(),
        "Midpoint_X": cluster["Midpoint_X"].median(),
        "Midpoint_Y": cluster["Midpoint_Y"].median(),
        "Top": cluster["Top"].min(),
        "Bottom": cluster["Bottom"].min(),
    }
    cluster_coords["Width"] = cluster_coords["Right"] - cluster_coords["Left"]
    return cluster_coords


def combine_row(row):
    return pd.Series([
        line.map(
            lambda x: x.sort_values(
                by="Left"
            ).reset_index(drop=True)["Text"].fillna("")
        ).agg(
            lambda x: " ".join(x.values)
        ) + " "
        for line in row
    ]).sum().str.strip()

col_spans = test_extractor.get_col_spans(test_words, TEST_PAGE)

col_spans

0                    (0.0, 0.1671319603919983)
1    (0.1671319603919983, 0.25465500354766846)
2    (0.25465500354766846, 0.3363614082336426)
3     (0.3363614082336426, 0.4708361327648163)
4     (0.4708361327648163, 0.6129726767539978)
5                    (0.6129726767539978, 1.0)
dtype: object

In [107]:
y_tol = table_words["Height"].max() * 1.5
x_tol = table_words["Width"].median()
sum_y_delta = y_tol
columnized = columnize(word_clusters[0], col_spans)
last_col_coords = pd.DataFrame.from_records(
    columnized.map(
        get_cluster_coords
    )
)
print(y_tol)
rows = []
current_row = [columnized]
print(" ".join(word_clusters[0].sort_values(by="Left")["Text"].values))
for count, cluster in enumerate(word_clusters[1:]):
    print("-"*50)
    print(" ".join(cluster.sort_values(by="Left")["Text"].values))
    columnized = columnize(cluster, col_spans)
    col_coords = pd.DataFrame.from_records(columnized.map(get_cluster_coords))
    nonempty = col_coords.dropna().index.to_series()
    last_nonempty = last_col_coords.dropna().index.to_series()
    delta_cols = (~nonempty.isin(last_nonempty)).any()
    y_delta = (
        col_coords["Midpoint_Y"].median()
        - last_col_coords["Midpoint_Y"].median()
    )
    sum_y_delta += y_delta
    mean_y_delta = sum_y_delta / (count + 2)
    min_y_delta = mean_y_delta*0.75
    print("Y Delta:", y_delta)
    print("Min Y Delta:", min_y_delta)
    print("More cols detected", delta_cols)
    print("Y tolerance exceeded", (y_delta > y_tol) and (y_delta > min_y_delta))
    if (delta_cols or (y_delta > y_tol)) and (y_delta > min_y_delta):
        combined_row = combine_row(current_row)
        print(combined_row)
        rows.append(combined_row)
        current_row = [columnized]
    else:
        current_row.append(columnized)
    last_col_coords = col_coords
print("-"*50)
combined_row = combine_row(current_row)
print(combined_row)
rows.append(combined_row)

0.007117736851796508
Central merica and the Fundraising 481
--------------------------------------------------
Carbbear
Y Delta: 0.004082381725311279
Min Y Delta: 0.00420004446641542
More cols detected False
Y tolerance exceeded False
--------------------------------------------------
Central merica the Grantmaking 303,788
Y Delta: 0.005004454404115677
Min Y Delta: 0.004051143245305866
More cols detected True
Y tolerance exceeded False
0    Central merica and the Carbbear
1                                   
2                                   
3                        Fundraising
4                                   
5                                481
dtype: object
--------------------------------------------------
Caribbean
Y Delta: 0.004541680216789246
Min Y Delta: 0.003889922474627383
More cols detected False
Y tolerance exceeded False
--------------------------------------------------
Central merica the Program services Academic Support 20,973
Y Delta: 0.004575680010020733
Min Y 

Y Delta: 0.004598274827003479
Min Y Delta: 0.0041470903318105755
More cols detected True
Y tolerance exceeded False
0    urope cluding Iceland Greenland)
1                                    
2                                    
3                       Prog Services
4                    Student Services
5                             321,429
dtype: object
--------------------------------------------------
Greenland
Y Delta: 0.004046544432640076
Min Y Delta: 0.004118572844443127
More cols detected False
Y tolerance exceeded False
--------------------------------------------------
Middle East and North Africa 1 2 Grantmaking 698,609
Y Delta: 0.005023092031478882
Min Y Delta: 0.0041097914989222765
More cols detected True
Y tolerance exceeded False
0    Europe :luding Icelan & Greenland
1                                     
2                                     
3                            vestments
4                                     
5                                     
dtype: obje

Y Delta: 0.0049121081829071045
Min Y Delta: 0.004425536290174732
More cols detected True
Y tolerance exceeded False
0    Russia & Newly Independer States
1                                    
2                                    
3                         Investments
4                                    
5                                    
dtype: object
--------------------------------------------------
South America Grantmaking 503,476
Y Delta: 0.008267834782600403
Min Y Delta: 0.004449527368509537
More cols detected False
Y tolerance exceeded True
0    South America
1                 
2                 
3      Fundraising
4                 
5            4,191
dtype: object
--------------------------------------------------
South America Progr Services Service centers 58,695
Y Delta: 0.00861252099275589
Min Y Delta: 0.004476325546856969
More cols detected True
Y tolerance exceeded True
0    South America
1                 
2                 
3      Grantmaking
4                 
5  

In [ ]:
rows[0][0]

In [ ]:
test_extractor.field_labels.map(
    lambda x: get_coordinate(test_extractor.tablemap, x, "Left", "Left_Default")
)

In [ ]:
word_clusters[1]

In [ ]:
columnize(word_clusters[1], col_spans).map(get_cluster_coords)

In [ ]:
test_extractor.tablemap

In [ ]:
header_words = test_extractor.get_header_words(test_words, 22)

In [ ]:
header_words.sort_values(by="Left")[["Text", "Left", "Right", "Midpoint_Y"]].head(50)

In [ ]:
pd.DataFrame(
    {
        "Gap": header_words.sort_values(by="Left").rolling(2).apply(lambda x: x),
        "Right": header_words.sort_values(by="Left")["Right"],
        "Text": header_words.sort_values(by="Left")["Text"],
    }
)

In [ ]:
def cluster_x(words, tolerance):
    print("Tolerance", tolerance)
    if (tolerance == 0) or (words.shape[0] < 2):
        return [
            [word] for (idx, word)
            in words.sort_values(by="Left").iterrows()
        ]
    groups = []
    sorted_words = words.sort_values(by="Left")
    current_group = [sorted_words.iloc[0]]
    last = sorted_words.iloc[0]["Right"]
    for idx, word in sorted_words.iloc[1:].iterrows():
        if word["Left"] <= (last + tolerance):
            current_group.append(word)
        else:
            print("New Group")
            print("*"*100)
            groups.append(current_group)
            current_group = [word]
        print("Left:", word["Left"])
        print("Right:", word["Right"])
        last = max((last, word["Right"]))
    groups.append(current_group)
    return [pd.DataFrame(group) for group in groups]

In [ ]:
left_right = pd.DataFrame(
    {
        "Left": header_words["Left"].sort_values().reset_index(drop=True),
        "Right": header_words["Right"].sort_values().reset_index(drop=True),
    }
)

header_lines = cluster_x(
    left_right, header_words["Width"].min()*.8
)

In [ ]:
len(header_lines)

In [ ]:
[
    (x["Left"].min(), x["Right"].max())
    for x in header_lines
]

In [ ]:
x_clusters = cluster_x(header_words, header_words["Width"].min()*.8)

In [ ]:
len(x_clusters)

In [ ]:
0.6793086230754852-0.6723970174789429

In [ ]:
[
    words.sort_values(by="Left")["Text"].agg(lambda x: " ".join(x.values))
    for words in x_clusters
]

In [ ]:
left_bounds = pd.Series(
        cluster["Left"].min() for cluster in x_clusters[1:]
)
right_bounds = pd.Series(
    [cluster["Right"].max() for cluster in x_clusters[:-1]]
)
offsets = right_bounds - left_bounds
full_left = pd.concat([pd.Series([0]), left_bounds + offsets]).reset_index(drop=True)
full_right = pd.concat([right_bounds - offsets, pd.Series([1])]).reset_index(drop=True)
col_spans = full_left.combine(full_right, lambda x, y: (x, y))
col_spans

In [ ]:
last_cluster_right = pd.concat([header_words, table_words])["Left"].min()
left_bounds = []
right_bounds = []
for cluster in x_clusters:
    print("Last Cluster Right", last_cluster_right)
    print("Cluster Left", cluster["Left"].min())
    print("Cluster Right", cluster["Right"].max())
    print("*"*50)
    left_bounds.append(last_cluster_right)
    last_cluster_right = max(
        cluster["Right"].max()
        + cluster["Left"].min()
        - last_cluster_right,
        cluster["Right"].max()
    )
    right_bounds.append(last_cluster_right)
col_spans = pd.Series(zip(left_bounds, right_bounds))
col_spans

In [ ]:
header_words["Midpoint_X"].round(2).value_counts()

In [ ]:
def count_crossing_lines(df, left, right):
    return (
        df.loc[
            df["Left"].between(left, right)
            & (df["Right"] > right),
            "Text"
        ].count()
        + df.loc[
            (df["Left"] < left)
            & df["Right"].between(left, right),
            "Text"
        ].count()
    )


def recalculate_intervals(left_interval, right_interval):
    left_left = left_interval[0]
    right_right = right_interval[1]
    total_span = right_right - left_left
    
    

In [ ]:
col_spans.map(
    lambda x: (x[1] + x[0]) / 2
)

In [ ]:
table_words.loc[
    table_words["Left"].between(0.12358373403549194, 0.3558424413204193)
    & (table_words["Midpoint_X"] > 0.3558424413204193),
    "Text"
]

In [ ]:
left_right = pd.DataFrame(
    {
        "left": header_words["Left"].sort_values().reset_index(drop=True),
        "right": header_words["Right"].sort_values().reset_index(drop=True),
    }
)
left_right.rolling(2).apply(lambda x: print(x, "\n" + "*"*20) or 1)

In [ ]:
left_right.head()

In [ ]:
1 and 2

In [ ]:
(test_words.groupby("Page")["Width"].max() / test_words.groupby("Page")["Height"].mean()).index.values

In [ ]:
pd.cut(header_words["Right"], 1000).unique().categories

In [ ]:
sorted_header_words = header_words.sort_values(by="Left")

In [ ]:
len(x_clusters)

In [ ]:
def cluster_x(words, tolerance):
    if (tolerance == 0) or (words.shape[0] < 2):
        return [
            [word] for (idx, word)
            in words.sort_values(by="Midpoint_X").iterrows()
        ]
    groups = []
    sorted_words = words.sort_values(by="Midpoint_X")
    current_group = [sorted_words.iloc[0]]
    last = sorted_words.iloc[0]["Midpoint_X"]
    for idx, word in sorted_words.iloc[1:].iterrows():
        last = (last + word["Midpoint_X"]) / 2
        if word["Midpoint_X"] <= (last + tolerance):
            current_group.append(word)
        else:
            print("New Group")
            print("*"*100)
            groups.append(current_group)
            current_group = [word]
            last = word["Right"]
        print("Word:", word["Text"])
        print("Left:", word["Left"])
        print("Right:", word["Right"])
    groups.append(current_group)
    return [pd.DataFrame(group) for group in groups]

In [ ]:
clusters = cluster_x(header_words, header_words["Width"].mean()*.7)

In [ ]:
len(clusters)

In [ ]:
header_words.sort_values(by="Midpoint_X").rolling(4)["Midpoint_X"].mean()

In [ ]:
left_clusters = cluster_words(header_words, header_words["Width"].mean()*.835, "Left")

In [ ]:
len(left_clusters)

In [ ]:
[
    cluster["Left"].mean() for cluster in left_clusters
]

In [ ]:
def get_col_spans(self, words, page):
    pass

In [ ]:
test_extractor.tablemap

In [ ]:
test_col_spans = pd.Series(
    [
        (0, 0.205227),
        (0.205227, 0.311407),
        (0.311407, 0.420936),
        (0.420936, 0.572975),
        (0.572975, 0.735998),
        (0.735998, 1),
    ]
)

In [ ]:
def find_new_right(df, right):
    return (
        df.loc[
            (df["Right"] > right*1.01)
            & (df["Left"] < right),
            "Left"
        ].min()
    )

In [ ]:
crossing_right = test_col_spans.map(
    lambda x: get_new_right(
        pd.concat([
            test_extractor.get_header_words(test_words, 18),
            test_extractor.get_table_words(test_words, 18),
        ]),
        x[1]
    )
)
crossing_right

In [ ]:
test_col_spans.where(
    crossing_right.isna(),
    test_col_spans.combine(crossing_right, lambda x, y: (x[0], y))
)

In [ ]:
test_col_spans

In [ ]:
init_left = test_extractor.field_labels.map(
    lambda x: get_coordinate(test_extractor.tablemap, x, "Left", "Left_Default")
)
init_right = pd.concat(
    [
        init_left.iloc[1:],
        pd.Series([1]),
    ],
    ignore_index=True,
)

In [ ]:
crossing_right = init_right.map(
    lambda x: find_new_right(
        pd.concat([
            test_extractor.get_header_words(test_words, 25),
            test_extractor.get_table_words(test_words, 25),
        ]),
        x
    )
)
crossing_right

In [ ]:
new_right = init_right.where(
    crossing_right.isna(),
    crossing_right,
)
new_right

In [ ]:
init_left

In [ ]:
init_left.iloc[1:] = new_right.iloc[:-1]

In [ ]:
init_left

In [ ]:
col_spans = init_left.combine(new_right, lambda x, y: (x, y))